In [ ]:
'''
Arquivos:
- Estabelecer a forma de guardar os arquivos conforme necessidade da CNN

Funções:
- Função de importação de arquivo com as seguintes colunas:
    Open High Low Close Volume + Indicadores
  Esta mesma função fará os índices das linhas serem as datas
- Função de escolha do intervalo de dias a terem o gráfico montado
- Função que gera o gráfico de candlestick com volume e indicadores

'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime
from datetime import timedelta
from PIL import Image

In [ ]:
'''
Importa os dados do ativo e renomeia as colunas
'''

file_name = 'BTOW3_1d.xlsx'
path = '../collected_ data/' + file_name

inplace = True
df_input = pd.read_excel(path)
df_input.dropna(inplace = inplace)
df_input.set_index('Data', inplace = inplace)
df_input.sort_index(axis = 0, inplace = inplace)
df_input.rename(inplace = inplace, columns = {
    'Abertura': 'Open',
    'Máxima': 'High',
    'Mínima': 'Low',
    'Fechamento': 'Close',
    'Volume Financeiro': 'Volume',
    'Média Móvel E [9]': 'MME009',
    'Média Móvel A [21]': 'MMA021',
    'Média Móvel A [200]': 'MMA200',
    'Média Móvel A [51]': 'MMA051',
    'Média Móvel A [80]': 'MMA080',
    'Média Móvel A [400]': 'MMA400',
    'Média Móvel E [400]': 'MME400'
})
df_input.head()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import timedelta strptime

In [ ]:
'''
Cria reordena as colunas dos indicadores em ordem alfabética
'''

df_candle = df_input[['Open', 'High', 'Low', 'Close', 'Volume']]
df_ind = df_input.reindex(sorted(df_input.columns[5:]), axis = 1)
df = pd.concat([df_candle, df_ind], axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
'''
Verifica se o dataframe reordenado é igual ao original
'''

columns = list(df.columns)
eq = True
for col in columns:
   eq = eq*df[col].equals(df_input[col])
if eq:
    print("Data Frame válido.")
else:
    print("Data Frame inválido!!!!")

In [ ]:
'''
Aprender a gerar os gráficos para então iterar os intervalos de data
'''

def plot_chart(df, start_date, end_date, add):
    df_plot = df.loc[start_date:end_date,:]
    erase_color = 'black'

    mc = mpf.make_marketcolors(up = 'g', down = 'r', wick = 'inherit', edge = 'inherit', volume = 'grey')
    s = mpf.make_mpf_style(marketcolors = mc, facecolor = erase_color, edgecolor = erase_color, figcolor = erase_color)
    addplot = mpf.make_addplot(df_plot[add])
    
    return mpf.plot(df_plot, type = 'candle', volume = True, style = s, addplot = addplot, savefig = '../generated_data/teste.png')

ind_columns = list(df.columns)[5:9]
start_date = '2007-01-01'
date_delta = 1000
end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days = date_delta)
end_date = end_date.strftime('%Y-%m-%d')

plot = plot_chart(df, start_date, end_date, ind_columns)

In [ ]:
# Implementar o crop na função plot_chat()
im = Image.open('../generated_data/teste.png')
box = (im.size[0]*0.202, im.size[1]*0.11, im.size[0]*0.88, im.size[1]*0.85)

# Para salvar em PNG
print(im.size)
im_crop = im.crop(box).save('../generated_data/teste_crop.png')

# Para salvar em JPG
# im = Image.open('../generated_data/teste.png').convert('RGB')
# im_crop = im.crop(box).save('../generated_data/teste_crop.jpg', quality = 95)